In [1]:
import pandas as pd

In [2]:
pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,0] # O penultimo numero é a melhor volta e o último numero é a pontuação do pole.
pontuacao_f1_sprint = [8,7,6,5,4,3,2,1,0,0,0,0]
jogo = "f1_2023"

if "f1_2022" in jogo:
    path_resultados = "Y://corrida_inscritos_2023.xls"
    opcao = 1
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2023.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2023.csv'
elif "f1_2023" in jogo:
    path_resultados = "Y://corrida_inscritos_2024.xls"
    opcao = 2
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2024.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2024.csv'
       

In [3]:
if opcao == 1:
    df_resultados = pd.read_excel(path_resultados, sheet_name="crossplay")
elif opcao == 2: 
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros")
elif opcao == 3:
    pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,2]
    df_resultados = pd.read_excel(path_resultados, sheet_name="f2")


In [4]:
df_resultados.drop(["Index"], axis=1, inplace=True)

In [5]:
df_resultados

,Belgica-S1,Belgica,Holanda,Italia,Singapura,Japão,Catar-S1,Catar,Texas-S1,Texas
0,Leonardo Menon,Leonardo Menon,Leonardo Menon,Muka,samuel_freitas26,Patrick,Alienobserv,Leonardo Menon,DarkLightBR,Leonardo Menon
1,samuel_freitas26,alissonzeira,Patrick,crazy-mars245,LucasBR21,LucasBR21,O_peralta99,LucasBR21,Leonardo Menon,BrPB Vitor
2,LucasBR21,Kuro,Kuro,Leonardo Menon,crazy-mars245,Leonardo Menon,LucasBR21,Alienobserv,BrPB Vitor,LucasBR21
3,Bill25binho,Zedatapioca,O_peralta99,samuel_freitas26,Savio-23BRA,Kuro,alissonzeira,Cristiano Roque,samuel_freitas26,Leozin_CRF
4,Savio-23BRA,Patrick,samuel_freitas26,Dinner,O_peralta99,O_peralta99,Fernando Vignola,TonyBR80,Leozin_CRF,DarkLightBR
5,Patrick,O_peralta99,Dinner,Kuro,alissonzeira,Alienobserv,Cristiano Roque,Sucla,Sucla,samuel_freitas26
6,Muka,Savio-23BRA,Alienobserv,Cristiano Roque,Kuro,alissonzeira,Ailton4514,Zedatapioca,O_peralta99,O_peralta99
7,Sucla,LucasBR21,matheus_fw02,Johnyangry,Patrick,TonyBR80,Zedatapioca,samuel_freitas26,LucasBR21,Sucla
8,Zedatapioca,Ailton4514,LucasBR21,TonyBR80,Fernando Vignola,samuel_freitas26,Leonardo Menon,alissonzeira,Ailton4514,Cristiano Roque
9,O_peralta99,Fernando Vignola,Muka,matheus_fw02,Muka,Cristiano Roque,Savio-23BRA,O_peralta99,Muka,Alienobserv


In [6]:
df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

In [7]:
for etapa in list(df_resultados.columns):
    df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
        df_etapa["Pontos"] = pontuacao_f1_sprint
    else:
        df_etapa["Pontos"] = pontuacao_f1
    df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

#### Gerando pontuação

In [8]:
try:
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferença"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao
except KeyError: #nao achou dnf
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferença"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao

#### Gera pontuação extendida

In [9]:
for etapa in list(df_resultados.columns):
    pilotos = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontuacao[etapa] = 0
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
        for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points
    else:
        for eachDriver, points in zip(pilotos, pontuacao_f1):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points

In [10]:
# Reposicionando as colunas
novasColunas = list(df_resultados.columns)
novasColunas.append("Pontos")
novasColunas.append("Diferença")
df_pontuacao = df_pontuacao[novasColunas]
df_pontuacao[:50]

,Belgica-S1,Belgica,Holanda,Italia,Singapura,Japão,Catar-S1,Catar,Texas-S1,Texas,Pontos,Diferença
Piloto,,,,,,,,,,,,
LEONARDO MENON,8,25,25,15,0,15,0,26,7,25,146,0
LUCASBR21,6,4,2,0,18,18,6,18,1,15,88,-58
SAMUEL_FREITAS26,7,0,10,12,25,2,0,4,5,8,73,-73
PATRICK,3,10,19,0,5,26,0,0,0,0,63,-83
KURO,0,15,15,8,6,12,0,0,0,0,56,-90
O_PERALTA99,0,8,12,0,10,10,7,1,2,6,56,-90
ALISSONZEIRA,0,18,0,0,8,6,5,2,0,0,39,-107
ALIENOBSERV,0,0,6,0,0,8,8,15,0,1,38,-108
CRAZY-MARS245,0,0,0,19,15,0,0,0,0,0,34,-112


In [11]:
df_pontuacao[51:]

,Belgica-S1,Belgica,Holanda,Italia,Singapura,Japão,Catar-S1,Catar,Texas-S1,Texas,Pontos,Diferença
Piloto,,,,,,,,,,,,


In [12]:
df_pontuacao[["Pontos", "Diferença"]][:50]

,Pontos,Diferença
Piloto,,
LEONARDO MENON,146,0
LUCASBR21,88,-58
SAMUEL_FREITAS26,73,-73
PATRICK,63,-83
KURO,56,-90
O_PERALTA99,56,-90
ALISSONZEIRA,39,-107
ALIENOBSERV,38,-108
CRAZY-MARS245,34,-112


In [13]:
df_pontuacao[["Pontos", "Diferença"]][51:]

,Pontos,Diferença
Piloto,,


In [14]:
if opcao == 1:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Inscritos exportada com sucesso...")
elif opcao == 2:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Membros exportada com sucesso...")

Pontuação da Corrida dos Membros exportada com sucesso...


### Descarte corrida dos Membros

In [15]:
# import numpy as np

# if opcao == 2:
#     k = 3
# else:
#     k = 5
    
# df_pontos2_agreg = pd.DataFrame()
# etapas_por_pais = {}
# corridas = []

# etapas = df_pontuacao.columns.tolist()
# etapas_split = [x.rsplit("-") for x in etapas]
# paises_unicos = list(set([x[0] for x in etapas_split]))


# for cadaPais in paises_unicos:
#     for cadaEtapa in etapas:
#         if cadaEtapa.startswith(cadaPais):
#             corridas.append(cadaEtapa)
#     etapas_por_pais[cadaPais] = corridas
#     corridas = []

# # deletar o país Pontos
# etapas_por_pais.pop("Pontos")

# pilotos = df_pontuacao.index.tolist()

# for cada_etapa_por_pais in etapas_por_pais:
#     for cada_piloto in pilotos:
#         pontuacao_total_piloto = 0
#         for cada_subetapa in etapas_por_pais[cada_etapa_por_pais]:
#             pontuacao_total_piloto += df_pontuacao.loc[cada_piloto, cada_subetapa]
#         df_pontos2_agreg.loc[cada_piloto, cada_etapa_por_pais] = pontuacao_total_piloto

# df_pontos2_agreg["pontos"] = df_pontuacao["Pontos"].copy()

# for index, row in df_pontos2_agreg.iterrows():
#     row_sorted = np.sort(row.values[:-1])
#     df_pontos2_agreg.loc[index, "pontos após descarte"] = np.sum(row_sorted[k:])
#     df_pontos2_agreg.loc[index, "pontos descartados"] = df_pontos2_agreg.loc[index, "pontos"] - df_pontos2_agreg.loc[index, "pontos após descarte"]



In [16]:
# if opcao == 2:
#     df_pontos2_agreg = df_pontos2_agreg[['Bahrain', 'Jeddah', 'Melbourne', 'Shanghai', 'Baku',	'Miami', 'Imola', 
#                       'Monaco', 'Espanha', 'Canadá', 'Austria', 'Inglaterra', 'pontos', 'pontos após descarte',
#                       'pontos descartados', 'Diferença']]
# else:
#     df_pontos2_agreg = df_pontos2_agreg[['Bahrain', 'Jeddah', 'Melbourne', 'Shanghai', 'Baku',	'Miami', 'Imola', 
#                       'Monaco', 'Espanha', 'Canada', 'Austria', 'Inglaterra', 'pontos', 'pontos após descarte',
#                       'pontos descartados', 'Diferença']]
    
# df_pontos2_agreg.sort_values(by="pontos após descarte", ascending=False)

### Calculo da pontuação mensal para o gift card  
Insira o intervalo das etapas equivalente ao mês que deseja calcular a pontuação.  
Exemplo: Se as etapas 3 e 4 forem as etapas realizadas no determinado mês, coloque entre colchetes '[2, 3]'

In [15]:
if opcao == 2:
    try:
        mes = {1: "Janeiro", 2: "Fevereiro", 3: "Março", 4:"Abril",
               5: "Maio", 6:"Junho", 7: "Julho", 8: "Agosto",
               9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"}
        etapa_inicio = 7
        etapa_fim = 12
        mes_code = 10
        df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
            if etapa.endswith("-S1"): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
                df_etapa["Pontos"] = pontuacao_f1_sprint
            else:
                df_etapa["Pontos"] = pontuacao_f1
            df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

        try:
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferença"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao
        except KeyError: #nao achou dnf
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferença"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            pilotos = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontuacao[etapa] = 0
            if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
                for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points
            else:
                for eachDriver, points in zip(pilotos, pontuacao_f1):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points

        # Reposicionando as colunas
        novasColunas = list(df_resultados.columns[etapa_inicio-1:etapa_fim])
        novasColunas.append("Pontos")
        novasColunas.append("Diferença")
        df_pontuacao = df_pontuacao[novasColunas]
        print(f"Pontuação dos membros do mês de {mes.get(mes_code)}")
        print(df_pontuacao[:50])
    except IndexError:
        print("Nenhuma etapa ainda no mês de {}".format(mes.get(mes_code)))
else:
    print("Pontuação mensal só é válida para a corrida dos membros!")

Pontuação dos membros do mês de Outubro
                  Catar-S1  Catar  Texas-S1  Texas Pontos Diferença
Piloto                                                             
LEONARDO MENON           0     26         7     25     58         0
LUCASBR21                6     18         1     15     40       -18
BRPB VITOR               0      0         6     18     24       -34
ALIENOBSERV              8     15         0      1     24       -34
DARKLIGHTBR              0      0         8     11     19       -39
CRISTIANO ROQUE          3     12         0      2     17       -41
SAMUEL_FREITAS26         0      4         5      8     17       -41
LEOZIN_CRF               0      0         4     12     16       -42
O_PERALTA99              7      1         2      6     16       -42
SUCLA                    0      8         3      4     15       -43
TONYBR80                 0     10         0      0     10       -48
ZEDATAPIOCA              1      6         0      0      7       -51
ALISSONZ